# SST-2
# BERT finetuning

## Librairy

In [2]:
import os
import time
import pickle

import numpy as np
import torch

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

from transformers import BertTokenizer, BertTokenizerFast
from transformers import BertForSequenceClassification, AdamW
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback
from transformers.data.data_collator import DataCollatorWithPadding

from datasets import load_dataset, Dataset, concatenate_datasets

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

## Global variables

In [4]:
BATCH_SIZE = 24 # cf. paper Sun et al.
NB_EPOCHS = 4   # cf. paper Sun et al.

In [5]:
# CURRENT_PATH = '~/Results/BERT_finetune' # put your path here
CURRENT_PATH = '/raid/home/jeremiec/Ax_results/BERT_finetune_v2'

In [6]:
RESULTS_FILE = os.path.join(CURRENT_PATH, 'sst-2_results.pkl')
RESULTS_DIR = os.path.join(CURRENT_PATH,'sst-2/')

In [7]:
# CACHE_DIR = '~/Data/huggignface/'         # put your path here
CACHE_DIR = '/raid/home/jeremiec/huggingface_datasets'

## Dataset

In [8]:
# download dataset

# train set
raw_datasets = load_dataset('glue', 'sst2', cache_dir=CACHE_DIR)
raw_datasets = raw_datasets.rename_column('sentence', 'text')

# special test set
raw_test_set = load_dataset('gpt3mix/sst2', split='test', cache_dir=CACHE_DIR)

def clean(example):
    example['text'] = example['text'].replace('-LRB-', '(').replace('-RRB-', ')').replace(r'\/', r'/')
    example['label'] = np.abs(example['label'] - 1) # revert labels of test set
    return example

raw_test_set = raw_test_set.map(clean)

raw_datasets['test'] = raw_test_set

Reusing dataset glue (/raid/home/jeremiec/huggingface_datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Using custom data configuration default
Reusing dataset sst2 (/raid/home/jeremiec/huggingface_datasets/gpt3mix___sst2/default/0.0.0/90167692658fa4abca2ffa3ede1a43a71e2bf671078c5c275c64c4231d5a62fa)
Loading cached processed dataset at /raid/home/jeremiec/huggingface_datasets/gpt3mix___sst2/default/0.0.0/90167692658fa4abca2ffa3ede1a43a71e2bf671078c5c275c64c4231d5a62fa/cache-3781b546d2425b83.arrow


In [9]:
# tokenize

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.rename_column('label', 'labels')
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

train_dataset = tokenized_datasets["train"].shuffle(seed=42)
test_dataset = tokenized_datasets["test"].shuffle(seed=42)
val_dataset = tokenized_datasets["validation"].shuffle(seed=42)

Loading cached processed dataset at /raid/home/jeremiec/huggingface_datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-4815fa44fbbbc915.arrow
Loading cached processed dataset at /raid/home/jeremiec/huggingface_datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-eacdae5f7dd705aa.arrow
Loading cached processed dataset at /raid/home/jeremiec/huggingface_datasets/gpt3mix___sst2/default/0.0.0/90167692658fa4abca2ffa3ede1a43a71e2bf671078c5c275c64c4231d5a62fa/cache-608f71b970febf5c.arrow
Loading cached shuffled indices for dataset at /raid/home/jeremiec/huggingface_datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-a54fff6093e96c07.arrow
Loading cached shuffled indices for dataset at /raid/home/jeremiec/huggingface_datasets/gpt3mix___sst2/default/0.0.0/90167692658fa4abca2ffa3ede1a43a71e2bf671078c5c275c64c4231d5a62fa/cache-1d9016119bcdeccf.arrow
Loading cached sh

In [10]:
# get number of labels

num_labels = len(set(train_dataset['labels'].tolist()))
num_labels

2

## Model

#### Model

In [11]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

#### Training

In [12]:
training_args = TrainingArguments(
    
    # output
    output_dir=RESULTS_DIR,          
    
    # params
    num_train_epochs=NB_EPOCHS,               # nb of epochs
    per_device_train_batch_size=BATCH_SIZE,   # batch size per device during training
    per_device_eval_batch_size=BATCH_SIZE,    # cf. paper Sun et al.
    learning_rate=2e-5,                       # cf. paper Sun et al.
#     warmup_steps=500,                         # number of warmup steps for learning rate scheduler
    warmup_ratio=0.1,                         # cf. paper Sun et al.
    weight_decay=0.01,                        # strength of weight decay
    
#     # eval
    evaluation_strategy="steps",              # cf. paper Sun et al.
    eval_steps=50, # 20                       # cf. paper Sun et al.
#     evaluation_strategy='no', # no more evaluation, takes time
    
    # log
    logging_dir=RESULTS_DIR+'logs',  
    logging_strategy='steps',
    logging_steps=50, # 20
    
    # save
    save_strategy='steps',
    save_total_limit=1,
    # save_steps=20, # default 500
    load_best_model_at_end=True,              # cf. paper Sun et al.
    metric_for_best_model='eval_loss'    
)

In [13]:
def compute_metrics(p):
    
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    
    return {"val_accuracy": accuracy}

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    # compute_metrics=compute_metrics,
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

In [15]:
results = trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, idx.
***** Running training *****
  Num examples = 67349
  Num Epochs = 4
  Instantaneous batch size per device = 24
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 1
  Total optimization steps = 11228


Step,Training Loss,Validation Loss
50,0.826500,0.819869
100,0.739300,0.676065
150,0.665800,0.613258
200,0.620600,0.494344
250,0.467300,0.372698
300,0.383100,0.352609
350,0.332300,0.321877
400,0.336800,0.307539
450,0.283600,0.301807
500,0.324200,0.282939


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, idx.
***** Running Evaluation *****
  Num examples = 872
  Batch size = 24
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, idx.
***** Running Evaluation *****
  Num examples = 872
  Batch size = 24
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, idx.
***** Running Evaluation *****
  Num examples = 872
  Batch size = 24
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, idx.
***** Running Evaluation *****
  Num examples = 872
  Batch size = 24
The following columns in the evaluation set  don't have a corresponding argument in `Ber

In [16]:
training_time = results.metrics["train_runtime"]
training_time_per_epoch = training_time / training_args.num_train_epochs
training_time_per_epoch

404.0919

In [17]:
trainer.save_model(os.path.join(RESULTS_DIR, 'checkpoint_best_model'))

Saving model checkpoint to /raid/home/jeremiec/Ax_results/BERT_finetune_v2/sst-2/checkpoint_best_model
Configuration saved in /raid/home/jeremiec/Ax_results/BERT_finetune_v2/sst-2/checkpoint_best_model/config.json
Model weights saved in /raid/home/jeremiec/Ax_results/BERT_finetune_v2/sst-2/checkpoint_best_model/pytorch_model.bin
tokenizer config file saved in /raid/home/jeremiec/Ax_results/BERT_finetune_v2/sst-2/checkpoint_best_model/tokenizer_config.json
Special tokens file saved in /raid/home/jeremiec/Ax_results/BERT_finetune_v2/sst-2/checkpoint_best_model/special_tokens_map.json


## Results

In [18]:
# load model
model_file = os.path.join(RESULTS_DIR, 'checkpoint_best_model')
finetuned_model = BertForSequenceClassification.from_pretrained(model_file, num_labels=num_labels)
finetuned_model.to(device)
finetuned_model.eval()

# compute test acc
test_trainer = Trainer(finetuned_model, data_collator=DataCollatorWithPadding(tokenizer))
raw_preds, labels, _ = test_trainer.predict(test_dataset)
preds = np.argmax(raw_preds, axis=1)
test_acc = accuracy_score(y_true=labels, y_pred=preds)

# save results
results_d = {}
results_d['accuracy'] = test_acc
results_d['training_time'] = training_time

loading configuration file /raid/home/jeremiec/Ax_results/BERT_finetune_v2/sst-2/checkpoint_best_model/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file /raid/home/jeremiec/Ax_results/BERT_finetune_v2/sst-2/checkpoint_best_model/pytorch_model.bin
All model checkpoi

In [19]:
results_d

{'accuracy': 0.9203734211971444, 'training_time': 1616.3676}

In [20]:
# save results

with open(RESULTS_FILE, 'wb') as fh:
    pickle.dump(results_d, fh)